<p style="color:dodgerblue; font-size: 31px; text-align:center;">
    <b>Getting Data</b>
</p>

<p style="color:black; font-size:25px;"><b>Plan:</b></p>

<p>       
1. Scraping info from <a href="https://www.seedtable.com/european-startup-rankings">European startups ranking</a>.
</p>
<p>
 2. Scraping info from <a href= 'https://www.crunchbase.com/'>Crunchbase</a>.
</p>
<p>  
 3. Storing data in a database.
</p>

In [1]:
#libraries
import pandas as pd

import time  #to set a random waiting time between requests 
import random 

import requests #to make requests
from bs4 import BeautifulSoup #to parse html response from request

<p style="color:DodgerBlue; font-size:23px;"><b> 1. European startups ranking </b></p>

<p>In this part we are going to take only startups from 7 popular countries in Europe:

<i><b>Germany, France, UK, Netherlands, Spain, Italy, and Switzerland</b></i> 
</p>


The data we are going to scrap about startups will be the following:



| Startup | Country | City | Description | Url |
|------|------|------|------|------|

<p>
    Where:
   
<b><i>Startup</i></b> - <i>startup name<i/> <br>
<b>Country</b> - <i>country where the startup was founded<i/> <br>
<b>City</b> - <i>city of the startup <i/> <br>
<b>Description</b> - <i>short description of the startup <i/> <br>
<b>Url</b> - <i>url of the startup<i/> <br>


    

In [2]:
list_of_countries = ['startups-germany',
                     'startups-france',
                     'startups-uk',
                     'startups-the-netherlands',
                     'startups-spain',
                     'startups-italy',
                     'startups-switzerland'
                    ]

d = {'Startup': [],
     'Country': [],
     'City': [],
     'Description': [],
     'Url': []
    }

In [3]:
def seedtable_scrap(list_of_countries):
    """Scraps data about top European startups given their countries"""
        
    url = 'https://www.seedtable.com/'
    
    for country in list_of_countries:
        
        r = requests.get(url + country)
    
        soup = BeautifulSoup(r.content)
        data = soup.find_all(id="company-horizontal-card")

        c = country.split('-')[-1].capitalize()

        for i in range(1, len(data) - 1):

            d['Country'].append(c)
            d['Startup'].append(data[i].a.text)
            d['City'].append(data[i].span.text.strip())
            d['Description'].append(data[i].p.text.strip())
            d['Url'].append(data[i].a.get('href'))

In [4]:
seedtable_scrap(list_of_countries)

In [5]:
startups_table = pd.DataFrame(d)

In [6]:
startups_table.head(5)

,Startup,Country,City,Description,Url
0,Delivery Hero,Germany,Berlin,Delivery Hero is a network of online food orde...,https://deliveryhero.com
1,N26,Germany,Berlin,N26 offers mobile banking solutions to custome...,https://n26.com
2,GetYourGuide,Germany,Berlin,GetYourGuide operates an online platform for b...,https://getyourguide.com/
3,Wirecard,Germany,Munich,Wirecard is fastest-growing digital platforms ...,https://wirecard.com/
4,CoachHub,Germany,Berlin,CoachHub is the digital coaching provider that...,https://coachhub.io/


<p style="color:DodgerBlue; font-size:23px;"><b> 2. Crunchbase </b></p>

<p>
    In order to get additional information about startups we are going to use<i>
    <a href="https://www.crunchbase.com/">Crunchbase</a> 
       (It is a platform for finding business information about private and public companies.) </i>
    

The data we are going to scrap about startups will be the following:


|Startup|Size|CB rank|Total funding|Last funding type|Investments|Acquisitions|Team members|Founded date|IPO status|Operating status|Company type|
|------|------|------|------|------|------|------|------|------|------|------|------|

<p>
    Where:
   
<b>Startup</b> - <i>startup name</i> <br>
<b>Size</b> - <i>number of people working in a company</i> <br>
<b>CB rank</b> - <i>rank of a company given by crunchbase</i> <br>
<b>Total funding</b> - <i>total amount of money company raised</i> <br>
<b>Last funding type</b> - <i>last funding type of a company</i> <br>
<b>Investments</b> - <i>number of investments made in other companies</i> <br>
<b>Acquisitions</b> - <i>number of companies acquired</i> <br>
<b>Team members</b> - <i>number of team members in crunchbase</i> <br>
<b>Founded date</b> - <i>date the company founded</i> <br>
<b>IPO status</b> - <i>"Private" or "Public"</i> <br>
<b>Operating status</b> - <i>"Active" or "Closed"</i> <br>
<b>Company type</b> - <i>"For profit" or "Non-profit"</i> <br>
<br><br>
    We are going to scrap additional data about industries that startup covers and put it into other dataframe to avoid repetition of startups in the main dataframe.<br> <i>(Because some startups can cover up to 10 industries, which will hugely increase the dataset)</i>


    

In [7]:
#define dictionary where will add the data
d_ = {'Startup': [],
     'Size': [],
     'CB rank': [],
     'Total funding': [],
     'Last funding type': [],
     'Investments': [],
     'Acquisitions': [],
     'Team members': [],
     'Founded date': [],
     'IPO status': [],
     'Operating status': [],
     'Company type': []
     }
industries = {'Startup': [],
             'Industry': []
             }

<p><i>
    The information in the HTML files is stored in tags.<br> We are going to use Beautiful Soup library to quickly and easily access information in those tags.
<br><br>
We define several functions that will scrap differently stored data:
</i></p>

In [8]:
def add_financial_info(soup):
    '''Adds company financial data (e.g. Total Funding Amount, Number of investors...)'''
    
    anv = soup.find('anchored-values') # financial data is stored in the tags named *anchored-values*
    
    if anv:
        for link in anv.find_all('a'):

            if link.span.span.text == 'Number of Acquisitions': 
                d_['Acquisitions'].append(link.find_all('span')[-1].text)

            elif link.span.span.text == 'Number of Investments':   
                d_['Investments'].append(link.find_all('span')[-1].text)

            elif link.span.span.text == 'Total Funding Amount': 
                d_['Total funding'].append(link.find_all('span')[-1].text)

            elif link.span.span.text == 'Number of Current Team Members': 
                d_['Team members'].append(link.find_all('span')[-1].text)

In [9]:
def add_general_info(soup):
    '''Adds general info about company (e.g. Founded Date, Operating Status, Num. of Employees...)'''
    
    li = soup.find_all('li')
    
    for _ in li:

        if _.a: #checks if _li_ tag contains _a_ tag

            if ('num_employees' in _.a.get('href')) & (len(d_['Size']) < len(d_['Startup'])):
                d_['Size'].append(_.a.text)

            elif ('last_funding_type' in _.a.get('href')) & (len(d_['Last funding type']) < len(d_['Startup'])):
                d_['Last funding type'].append(_.a.text)

            elif ('rank_org_company' in _.a.get('href')) & (len(d_['CB rank']) < len(d_['Startup'])): 
                d_['CB rank'].append(_.a.text)


        elif (_.span and _.span.span): #checks if _li_ tag contains _span_ tag
                                        # and if _span_ tag also contains one more span tag
            if _.span.span.text == 'Founded Date': d_['Founded date'].append(_.find_all('span')[-1].text)   #append(_.find(contexttype="profile").text)

            elif _.span.span.text == 'Operating Status': d_['Operating status'].append(_.find_all('span')[-1].text)  #append(_.find(contexttype="profile").text)

            elif _.span.span.text == 'Company Type': d_['Company type'].append(_.find_all('span')[-1].text)   #append(_.find(contexttype="profile").text)

    
    d_['IPO status'].append('Public' if 'IPO' in d_['Last funding type'][-1] else 'Private')

<p><i>
In case there is no data about startup, we are going to add <b>Nan</b> values there.
</i></p>

In [10]:
def add_nan_values():
    '''Checks dictionary and adds Nan values in places where there was no data in website'''
    
    t = len(d_['Startup'])
    
    for k,v in d_.items():
#         print(k,v)
        if len(v) < t:
            d_[k].append('')

In [11]:
def add_industries(soup,startup_name):
    '''Adds industries to their corresponding startups'''
    
    #the industry data is stored in tags with attribute 'mat-chip'
    for ind in soup.find_all('mat-chip'): 
        
        industries['Startup'].append(startup_name)
        industries['Industry'].append(ind.text) 

<p><i>
    With high number of requests we can be blocked from the website. <br>
    <br>To avoid this we define a function that will make requests with a random time interval between each of the requests and also additional random time interval for each 9 requests we make. 
    
   This will allow us to gently proceed with scraping. 
<br> </i></p> 

In [12]:
def get_request(url, headers = None, counter = [0]):
    '''Making requests with a random time interval between each.'''
    
    counter[0] += 1
    
    if counter[0] % 9 == 0:
        sleep_time = random.randint(120, 180) #random interval between 2-3 minutes each 9 requests
    else: 
        sleep_time = random.randint(45, 75) #random interval between 45-75 seconds with each request
    
    session = requests.Session()
    
    time.sleep(sleep_time) #make sleep
    
    r = session.get(url, headers = headers) #make request
    
    if r.ok: #check if server response is ok, else raise error
        return r
    else:
        r.raise_for_status()

<p><i>There are several startups in Crunchbase that have the same name. <br> <br>In order to get the data about our startup, we are going to use startups' urls as a unique identifier of the company, and search for this information in crunchbase using Google </i> </p>

In [13]:
def getting_address_from_google(startup_website):
    '''Searches for startup's Crunchbase profile in google given its url'''
    
    gsearch_url = 'https://www.google.com/search?q={}+crunchbase'.format(startup_website)
    
    r = requests.get(gsearch_url)  # request to google search's result page
    
    soup = BeautifulSoup(r.content,'html.parser')
    
    link = soup.find_all('div',class_ = 'kCrYT')[0].a.get('href')# for each result of Google I get 2 items.
                                                    # 1-Headline (containing hyperlink to the website)
                                                    # 2-Short description of the Result
        
# Thus as default google gives 10 results, soup.find_all('div',class_ = 'kCrYT') will give list of
# 20 items and the first one will be the Headline of first result containing the hyperlink.
    
    start = link.find('=') + 1  #cleaning the link
    end = link.find('&') 
    
    return link[start:end]  #returns link of this startup's profile on crunchbase

In [14]:
def parsing_data_from_crunchbase(df):
    '''Takes url of startup's Crunchbase profile and parses data from Crunchbase
    
    -------------
    df - DataFrame containing Startup name and its Url.
    -------------
    '''
    for s,u in df.loc[:,['Startup','Url']].itertuples(index = False):

        s_num = df.loc[df.loc[:,'Startup'] == s ].index[0]
        
        print('Now on startup number {} --> {}'.format(s_num,s))
        print('Number of startups left: {}\n'.format(df.index[-1] - s_num))
        
        link = getting_address_from_google(u)
    
        #header to behave as browser
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

        r = get_request(link, headers)

        soup = BeautifulSoup(r.content,'html.parser')

        d_['Startup'].append(s)

        add_general_info(soup)
        add_financial_info(soup)
        add_industries(soup, s)

        add_nan_values()


<p><b>Now let's scrap the data!</b></p>

In [18]:
parsing_data_from_crunchbase(startups_table)

Now on startup number 0 --> Delivery Hero
Number of startups left: 846

Now on startup number 1 --> N26
Number of startups left: 845

Now on startup number 2 --> GetYourGuide
Number of startups left: 844

Now on startup number 3 --> Wirecard
Number of startups left: 843

Now on startup number 4 --> CoachHub
Number of startups left: 842

Now on startup number 5 --> SoundCloud
Number of startups left: 841

Now on startup number 6 --> Ascent
Number of startups left: 840

Now on startup number 7 --> Lilium
Number of startups left: 839

Now on startup number 8 --> Mambu
Number of startups left: 838

Now on startup number 9 --> Raisin
Number of startups left: 837

Now on startup number 10 --> TIER
Number of startups left: 836

Now on startup number 11 --> Volocopter
Number of startups left: 835

Now on startup number 12 --> IDnow
Number of startups left: 834

Now on startup number 13 --> Helpling
Number of startups left: 833

Now on startup number 14 --> HelloFresh
Number of startups left: 8

Now on startup number 172 --> Actility
Number of startups left: 674

Now on startup number 173 --> Kayrros
Number of startups left: 673

Now on startup number 174 --> ImCheck Therapeutics
Number of startups left: 672

Now on startup number 175 --> JobTeaser
Number of startups left: 671

Now on startup number 176 --> 360Learning
Number of startups left: 670

Now on startup number 177 --> OVH
Number of startups left: 669

Now on startup number 178 --> FINALCAD
Number of startups left: 668

Now on startup number 179 --> Sinequa
Number of startups left: 667

Now on startup number 180 --> EasyMile
Number of startups left: 666

Now on startup number 181 --> NAVYA
Number of startups left: 665

Now on startup number 182 --> Sanofi Pasteur
Number of startups left: 664

Now on startup number 183 --> Kard
Number of startups left: 663

Now on startup number 184 --> Delair
Number of startups left: 662

Now on startup number 185 --> Shine
Number of startups left: 661

Now on startup number 186 --> i

Now on startup number 315 --> Carwow
Number of startups left: 531

Now on startup number 316 --> Thought Machine
Number of startups left: 530

Now on startup number 317 --> Yoyo Wallet
Number of startups left: 529

Now on startup number 318 --> MADE
Number of startups left: 528

Now on startup number 319 --> iwoca
Number of startups left: 527

Now on startup number 320 --> Chip Financial
Number of startups left: 526

Now on startup number 321 --> ComplyAdvantage
Number of startups left: 525

Now on startup number 322 --> Ring
Number of startups left: 524

Now on startup number 323 --> Access Alto
Number of startups left: 523

Now on startup number 324 --> Blippar
Number of startups left: 522

Now on startup number 325 --> Brandwatch
Number of startups left: 521

Now on startup number 326 --> Bulb
Number of startups left: 520

Now on startup number 327 --> Trussle
Number of startups left: 519

Now on startup number 328 --> Tractable
Number of startups left: 518

Now on startup number 32

Now on startup number 456 --> Gameye
Number of startups left: 390

Now on startup number 457 --> Wonderkind
Number of startups left: 389

Now on startup number 458 --> Tradefox
Number of startups left: 388

Now on startup number 459 --> Lendahand
Number of startups left: 387

Now on startup number 460 --> Lone Rooftop
Number of startups left: 386

Now on startup number 461 --> CoolGames
Number of startups left: 385

Now on startup number 462 --> Wercker
Number of startups left: 384

Now on startup number 463 --> Deskbookers
Number of startups left: 383

Now on startup number 464 --> Airchip
Number of startups left: 382

Now on startup number 465 --> Cybersprint
Number of startups left: 381

Now on startup number 466 --> SemioticLabs
Number of startups left: 380

Now on startup number 467 --> Tykn
Number of startups left: 379

Now on startup number 468 --> EcoChain Technologies
Number of startups left: 378

Now on startup number 469 --> Talmundo
Number of startups left: 377

Now on star

Now on startup number 599 --> Verbio
Number of startups left: 247

Now on startup number 600 --> Aplazame
Number of startups left: 246

Now on startup number 601 --> Equidam
Number of startups left: 245

Now on startup number 602 --> beWanted
Number of startups left: 244

Now on startup number 603 --> ApetEat
Number of startups left: 243

Now on startup number 604 --> RatedPower
Number of startups left: 242

Now on startup number 44 --> Penta
Number of startups left: 802

Now on startup number 606 --> Satispay
Number of startups left: 240

Now on startup number 607 --> Supermercato24
Number of startups left: 239

Now on startup number 608 --> Delos
Number of startups left: 238

Now on startup number 609 --> Uala
Number of startups left: 237

Now on startup number 610 --> Musement
Number of startups left: 236

Now on startup number 611 --> D-Orbit
Number of startups left: 235

Now on startup number 612 --> Eligo
Number of startups left: 234

Now on startup number 613 --> Credimi
Number 

Now on startup number 773 --> Hosco
Number of startups left: 73

Now on startup number 774 --> RetinAI Medical
Number of startups left: 72

Now on startup number 775 --> Cosmos Network
Number of startups left: 71

Now on startup number 776 --> B3i
Number of startups left: 70

Now on startup number 777 --> Crypto Finance
Number of startups left: 69

Now on startup number 778 --> Akselos
Number of startups left: 68

Now on startup number 779 --> Advanon
Number of startups left: 67

Now on startup number 780 --> GenomSys
Number of startups left: 66

Now on startup number 781 --> MaxiVAX
Number of startups left: 65

Now on startup number 782 --> Verity Studios
Number of startups left: 64

Now on startup number 783 --> Ambrosus
Number of startups left: 63

Now on startup number 784 --> SMART VALOR
Number of startups left: 62

Now on startup number 785 --> Hydromea
Number of startups left: 61

Now on startup number 786 --> Knip
Number of startups left: 60

Now on startup number 787 --> GTX M

<p>
    <b>
    After scraping all the data, we are converting dictionaries with our data into pandas dataframes.
    </b>
</p>

In [26]:
cb = pd.DataFrame(d_)

In [27]:
cb.sort_values('Startup', inplace= True)

In [28]:
cb.head()

,Startup,Size,CB rank,Total funding,Last funding type,Investments,Acquisitions,Team members,Founded date,IPO status,Operating status,Company type
685,200crowd,11-50,"39,694",€300K,Seed,NaN,NaN,7.0,2015,Private,Active,For Profit
663,24 Ore Business School,1001-5000,"141,808",NaN,NaN,NaN,NaN,2.0,"Nov 9, 1965",Private,Active,For Profit
442,24sessions,11-50,"14,516",$1.5M,Series A,NaN,NaN,6.0,"Oct 1, 2015",Private,Active,For Profit
445,30MHz,11-50,"17,051",€4.9M,Series A,NaN,NaN,4.0,2014,Private,Active,For Profit
175,360Learning,101-250,"3,300",$42.6M,Series B,NaN,NaN,6.0,2009,Private,Active,For Profit


In [30]:
ind = pd.DataFrame(industries)

In [31]:
ind.sort_values('Startup', inplace= True)

In [32]:
ind.head(7)

,Startup,Industry
0,200crowd,Crowdfunding
1,200crowd,Financial Services
2,200crowd,FinTech
3,200crowd,Funding Platform
4,24 Ore Business School,News
5,24 Ore Business School,Publishing
6,24sessions,SaaS


<p>Now let's merge DataFrames <b><i>startups_table</i></b> and <b><i>cb</i></b>
.</p>

In [33]:
startups_table.sort_values('Startup', inplace = True)

In [34]:
startups_table.head(1)

,Startup,Country,City,Description,Url
688,200crowd,Italy,Milan,200 Crowd is a Crowdfunding platform that conn...,https://200crowd.com


In [35]:
cb.head(1)

,Startup,Size,CB rank,Total funding,Last funding type,Investments,Acquisitions,Team members,Founded date,IPO status,Operating status,Company type
685,200crowd,11-50,"39,694",€300K,Seed,NaN,NaN,7.0,2015,Private,Active,For Profit


<p style="color:black">
    <i>
        <b>1-st: </b>Sorted 2 DataFrames on Startup name.
   <b>
    2-nd: 
   </b>
      Merged 2 dataframes on index. 
        
In 2nd case it left two 'Startup' columns from first and second df as 'Startup_x' and 'Startup_y', we are going to drop 'Startup_y' and rename the remaining one as 'Startup'.</i> </p>

In [36]:
df = pd.merge(cb, startups_table, left_index=True, right_index=True) 

In [37]:
df.drop(columns = ['Startup_y'], inplace = True )

In [38]:
df.rename(columns = {'Startup_x': 'Startup'}, inplace = True)

In [39]:
df.head()

,Startup,Size,CB rank,Total funding,Last funding type,Investments,Acquisitions,Team members,Founded date,IPO status,Operating status,Company type,Country,City,Description,Url
685,200crowd,11-50,"39,694",€300K,Seed,NaN,NaN,7.0,2015,Private,Active,For Profit,Italy,Milan,nCore HR is a modern video ATS/Recruiting & Vi...,https://ncorehr.com/en/
663,24 Ore Business School,1001-5000,"141,808",NaN,NaN,NaN,NaN,2.0,"Nov 9, 1965",Private,Active,For Profit,Italy,Rome,Enel is a multi-national power company special...,https://enel.com/en-GB
442,24sessions,11-50,"14,516",$1.5M,Series A,NaN,NaN,6.0,"Oct 1, 2015",Private,Active,For Profit,Netherlands,Rotterdam,Helloprint is Europe's fastest growing online ...,https://helloprint.com
445,30MHz,11-50,"17,051",€4.9M,Series A,NaN,NaN,4.0,2014,Private,Active,For Profit,Netherlands,Amsterdam,Unless is a predictive personalization platfor...,https://unless.com
175,360Learning,101-250,"3,300",$42.6M,Series B,NaN,NaN,6.0,2009,Private,Active,For Profit,France,Paris,JobTeaser is a European job board platform for...,https://corporate.jobteaser.com/fr


<p>
    <b>
            We are done with our data. 
            <br>
            <br>
            Now we are going to store those Dataframes in PostgreSQL Database.
    </b>
    
</p>

<p style="color:DodgerBlue; font-size:23px;"><b> 3. Storing data in a database.</b></p>

<p>
    To store our data in DB, we are going to use the <b>psycopg2</b> which is PostgreSQL database adapter for Python.
    <br>
    This will allow us to make database operations through python.
</p>

In [40]:
import psycopg2

In [41]:
#connecting to DB
conn = psycopg2.connect("dbname=db user=user password=password")

In [42]:
#creating cursor to make DB operations
cur = conn.cursor()

<p>Creating table for the <b>main</b> DataFrame</p>

In [43]:
table_main = '''
CREATE TABLE main(
Startup VARCHAR PRIMARY KEY,
Size VARCHAR,
"CB rank" VARCHAR,
"Total funding" VARCHAR,
"Last funding type" VARCHAR,
"Investments" NUMERIC,
"Acquisitions" NUMERIC,
"Team members" NUMERIC,
"Founded date" VARCHAR,
"IPO status" VARCHAR(7),
"Operating status" CHAR(6),
"Company type" CHAR(10),
Country VARCHAR,
City VARCHAR,
Desctiption VARCHAR,
Url VARCHAR
);
''' 
#executing command in DB
cur.execute(table_main)

<p>Adding values of the <b>main</b> table</p>

In [46]:
values = ','.join(['%s' for c in df.columns])   #making placeholders for our data

In [47]:
values

'%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s'

In [48]:
query = "INSERT INTO main VALUES ({});".format(values)

In [49]:
for v in df.values: #inserting values one by one
    cur.execute(query, v)

In [50]:
conn.commit() #saving changes to DB

<p>Let's see our data in DB</p>

In [51]:
cur.execute('SELECT * FROM main;') 

In [52]:
for row in cur.fetchmany(3): print(row, '\n')

('9TLabs', '11-50', '10,788', '$5.5M', 'Seed', Decimal('NaN'), Decimal('NaN'), Decimal('3.0'), '2018', 'Private', 'Active', 'For Profit', 'Switzerland', 'Zürich', '9T Labs advances lightweight design by simplifying the use of high-performance materials.', 'https://9tlabs.com/') 

('ABA English', '51-100', '8,814', '$15.4M', 'Venture - Series Unknown', Decimal('NaN'), Decimal('NaN'), Decimal('5.0'), '2007', 'Private', 'Active', 'For Profit', 'Spain', 'Barcelona', 'ABA English offers a unique teaching methodology and uses its own proprietary technology.', 'https://abaenglish.com/en/') 

('ABB', '10001+', '1,418', '$2.7M', 'Grant', Decimal('5.0'), Decimal('31.0'), Decimal('63.0'), 'Jan 5, 1988', 'Private', 'Active', 'For Profit', 'Switzerland', 'Zürich', 'ABB provides power and automation technologies for smart grids, robotics, electric cars, renewable energy and motors.', 'https://abb.com') 


<p>
   In <b>DF</b> DataFrame - <b><i>Startup</i></b> is a primary key.
    
    
   In <b>IND</b> DataFrame - <b><i>Startup</i></b> is a foreign key and references <b><i>Startup</i></b> in <b>DF DataFrame</b>.

Primary keys of <b>IND</b> DataFrame are <i><b>Startup</b></i> and <i><b>Industry</b></i></p>

In [53]:
table_ind = '''
CREATE TABLE industry(
Startup VARCHAR,
Industry VARCHAR,
PRIMARY KEY(Startup, Industry),
FOREIGN KEY(Startup)
    REFERENCES main(Startup)
    ON DELETE CASCADE
);
'''

In [54]:
cur.execute(table_ind) #execute query

<p><i>Adding values of the <b>industry</b> table</i></p>

In [55]:
values_ = ','.join(['%s' for c in ind.columns])

In [57]:
values_

'%s,%s'


In [58]:
query_ = 'INSERT INTO industry VALUES ({});'.format(values_)

In [59]:
for v in ind.values:
    cur.execute(query_, v)

In [60]:
conn.commit() #saving changes to db

In [62]:
cur.execute('SELECT * FROM industry')

In [63]:
for row in cur.fetchmany(7): print(row, '\n')

('200crowd', 'Crowdfunding') 

('200crowd', 'Financial Services') 

('200crowd', 'FinTech') 

('200crowd', 'Funding Platform') 

('24 Ore Business School', 'News') 

('24 Ore Business School', 'Publishing') 

('24sessions', 'SaaS') 



In [64]:
cur.close() 
conn.close() #closing connection with db

<p style="color:dodgerblue; font-size:25px"><b>Now we have finished with the first part of getting the data and storing it in a database. <br> <br>
In the second part to we are going to clean the data and perform EDA.<b> </p>